In [1]:
DATADIR = '.' # unzipped train and test data
OUTDIR = './model-k' # just a random name
# Data Loading
!sudo rm -r ~/notebook/model-k
import os, shutil
import os
import re
from glob import glob
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

SELECTED_FOLD = 4

In [2]:
!sudo rm -r ~/notebook/model-k

rm: cannot remove '/home/prune/notebook/model-k': No such file or directory


In [3]:
AUDIO_LENGTH = 16000

def tf_fix_pad_transform(wave, length=AUDIO_LENGTH):
    # wave = np.pad(wave, (0, max(0, AUDIO_LENGTH - len(wave))), 'constant')
    # return wave

    if len(wave)<AUDIO_LENGTH:
        L = abs(len(wave)-AUDIO_LENGTH)
        start = L//2
        wave  = np.pad(wave, (start, L-start), 'constant')

    elif len(wave)>AUDIO_LENGTH:
        L = abs(len(wave)-AUDIO_LENGTH)
        start = L//2
        wave  = wave[start: start+AUDIO_LENGTH]

    return wave

In [4]:
POSSIBLE_LABELS = 'yes no up down left right on off stop go silence unknown'.split()
id2name = {i: name for i, name in enumerate(POSSIBLE_LABELS)}
name2id = {name: i for i, name in id2name.items()}


def load_data(data_dir):
    """ Return 2 lists of tuples:
    [(class_id, user_id, path), ...] for train
    [(class_id, user_id, path), ...] for validation
    """
    # Just a simple regexp for paths with three groups:
    # prefix, label, user_id
    pattern = re.compile("(.+\/)?(\w+)\/([^_]+)_.+wav")
    all_files = glob(os.path.join(data_dir, 'train/audio/*/*wav'))

    with open(os.path.join(data_dir, 'train/validation_list.txt'), 'r') as fin:
        validation_files = fin.readlines()
    valset = set()
    for entry in validation_files:
        r = re.match(pattern, entry)
        if r:
            valset.add(r.group(3))

    possible = set(POSSIBLE_LABELS)
    train, val = [], []
    for entry in all_files:
        r = re.match(pattern, entry)
        if r:
            label, uid = r.group(2), r.group(3)
            if label == '_background_noise_':
                label = 'silence'
            if label not in possible:
                label = 'unknown'

            label_id = name2id[label]

            sample = (label_id, uid, entry)
            if uid in valset:
                val.append(sample)
            else:
                train.append(sample)

    print('There are {} train and {} val samples'.format(len(train), len(val)))
    return train, val

trainset, valset = load_data(DATADIR)

There are 60281 train and 6798 val samples


In [5]:
import pandas as pd

folds = pd.read_csv('folds_by_ven.csv')

POSSIBLE_LABELS = 'yes no up down left right on off stop go silence unknown'.split()
id2name = {i: name for i, name in enumerate(POSSIBLE_LABELS)}
name2id = {name: i for i, name in id2name.items()}

valset = []
val_fold = folds[folds.fold==SELECTED_FOLD]
for i in range(len(folds[folds.fold==SELECTED_FOLD])):
    
    x = (name2id[val_fold.label.values[i]],val_fold.path.values[i].split('/')[-1].split('_')[0],val_fold.path.values[i])
    valset.append(x)
    
trainset = []
val_fold = folds[folds.fold!=SELECTED_FOLD]
for i in range(len(folds[folds.fold!=SELECTED_FOLD])):
    
    x = (name2id[val_fold.label.values[i]],val_fold.path.values[i].split('/')[-1].split('_')[0],val_fold.path.values[i])
    trainset.append(x)


In [6]:
import numpy as np
from scipy.io import wavfile

def data_generator(data, params, mode='train'):
    def generator():
        if mode == 'train':
            np.random.shuffle(data)
        # Feel free to add any augmentation
        for (label_id, uid, fname) in data:
            try:
                _, wav = wavfile.read(fname)
                wav = wav.astype(np.float32) / np.iinfo(np.int16).max

                L = 16000  # be aware, some files are shorter than 1 sec!
                if len(wav) < L:
                    continue
                # let's generate more silence!
                samples_per_file = 1 if label_id != name2id['silence'] else 20
                for _ in range(samples_per_file):
                    if len(wav) > L:
                        beg = np.random.randint(0, len(wav) - L)
                    else:
                        beg = 0
                    yield dict(
                        target=np.int32(label_id),
                        wav=wav[beg: beg + L],
                    )
            except Exception as err:
                print(err, label_id, uid, fname)

    return generator

In [7]:
import tensorflow as tf
from tensorflow.contrib import layers

def baseline(x, params, is_training):
    #x = layers.batch_norm(x, is_training=is_training)
    for i in range(13+1):
        y = layers.conv2d(x, 45, kernel_size=(3,3), activation_fn=tf.nn.relu,rate = int(2**(i // 3)))
        if i ==0:
            old_x = y
        if i > 0 and i % 2 == 0:
            x = y + old_x
            old_x = x
        else:
            x = y
        if i>0:
            x = layers.batch_norm(x)       
    x = tf.reduce_max(x, axis=[1, 2], keep_dims=True)
    x = layers.conv2d(x, 128, 1, 1, activation_fn=tf.nn.elu)
    x = tf.nn.dropout(x, keep_prob=params.keep_prob if is_training else 1.0)
    logits = layers.conv2d(x, params.num_classes, 1, 1, activation_fn=None)
   # x = tf.reshape(x,shape = (x.shape[0],x.shape[1],-1))
    #x = tf.reduce_mean(x, axis=[2])#, keep_dims=True)
    #x = layers.linear(x, 45,activation_fn=tf.nn.relu)
    #logits = layers.linear(x, params.num_classes)
    return tf.squeeze(logits, [1, 2])



In [8]:
from tensorflow.contrib import signal

# features is a dict with keys: tensors from our datagenerator
# labels also were in features, but excluded in generator_input_fn by target_key

def model_handler(features, labels, mode, params, config):
    # Im really like to use make_template instead of variable_scopes and re-usage
    extractor = tf.make_template(
        'extractor', baseline,
        create_scope_now_=True,
    )
    # wav is a waveform signal with shape (16000, )
    wav = features['wav']
    # we want to compute spectograms by means of short time fourier transform:
    specgram = signal.stft(
        wav,
        400,  # 16000 [samples per second] * 0.025 [s] -- default stft window frame
        160#00,  # 16000 * 0.010 -- default stride
    )
    # specgram is a complex tensor, so split it into abs and phase parts:
    phase = tf.angle(specgram) / np.pi
    # log(1 + abs) is a default transformation for energy units
    amp = tf.log1p(tf.abs(specgram))
    
    x = tf.stack([amp, phase], axis=3) # shape is [bs, time, freq_bins, 2]
    x = tf.to_float(x)  # we want to have float32, not float64

    logits = extractor(x, params, mode == tf.estimator.ModeKeys.TRAIN)

    if mode == tf.estimator.ModeKeys.TRAIN:
        loss = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits))
        # some lr tuner, you could use move interesting functions
        def learning_rate_decay_fn(learning_rate, global_step):
            return tf.train.exponential_decay(
                learning_rate, global_step, decay_steps=10000, decay_rate=0.99)

        train_op = tf.contrib.layers.optimize_loss(
            loss=loss,
            global_step=tf.contrib.framework.get_global_step(),
            learning_rate=params.learning_rate,
            optimizer=lambda lr: tf.train.AdamOptimizer(),#(lr, 0.9, use_nesterov=True),
            learning_rate_decay_fn=learning_rate_decay_fn,
            clip_gradients=params.clip_gradients,
            variables=tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES))

        specs = dict(
            mode=mode,
            loss=loss,
            train_op=train_op,
        )

    if mode == tf.estimator.ModeKeys.EVAL:
        prediction = tf.argmax(logits, axis=-1)
        acc, acc_op = tf.metrics.mean_per_class_accuracy(
            labels, prediction, params.num_classes)
        loss = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits))
        specs = dict(
            mode=mode,
            loss=loss,
            eval_metric_ops=dict(
                acc=(acc, acc_op),
            )
        )

    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {
            'label': tf.nn.softmax(logits),  # for probability just take tf.nn.softmax()
            'sample': features['sample'], # it's a hack for simplicity
        }
        specs = dict(
            mode=mode,
            predictions=predictions,
        )
    return tf.estimator.EstimatorSpec(**specs)


def create_model(config=None, hparams=None):
    return tf.estimator.Estimator(
        model_fn=model_handler,
        config=config,
        params=hparams,
    )

In [9]:
params=dict(
    seed=2018,
    batch_size=64,
    keep_prob=0.5,
    learning_rate=1e-3,
    clip_gradients=15.0,
    use_batch_norm=True,
    num_classes=len(POSSIBLE_LABELS)
)

hparams = tf.contrib.training.HParams(**params)
os.makedirs(os.path.join(OUTDIR, 'eval'), exist_ok=True)
model_dir = OUTDIR

run_config = tf.contrib.learn.RunConfig(model_dir=model_dir,keep_checkpoint_max=40)


In [10]:
# it's a magic function :)
from tensorflow.contrib.learn.python.learn.learn_io.generator_io import generator_input_fn
            
train_input_fn = generator_input_fn(
    x=data_generator(trainset, hparams, 'train'),
    target_key='target',  # you could leave target_key in features, so labels in model_handler will be empty
    batch_size=hparams.batch_size, shuffle=True, num_epochs=None,
    queue_capacity=3 * hparams.batch_size + 10, num_threads=1,
)

val_input_fn = generator_input_fn(
    x=data_generator(valset, hparams, 'val'),
    target_key='target',
    batch_size=hparams.batch_size, shuffle=True, num_epochs=None,
    queue_capacity=3 * hparams.batch_size + 10, num_threads=1,
)
            

def _create_my_experiment(run_config, hparams):
    exp = tf.contrib.learn.Experiment(
        estimator=create_model(config=run_config, hparams=hparams),
        train_input_fn=train_input_fn,
        eval_input_fn=val_input_fn,
        train_steps=10000, # just randomly selected params
        eval_steps=250,  # read source code for steps-epochs ariphmetics
        train_steps_per_iteration=500,
    )
    return exp

tf.contrib.learn.learn_runner.run(
    experiment_fn=_create_my_experiment,
    run_config=run_config,
    schedule="continuous_train_and_eval",
    hparams=hparams)

INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_evaluation_master': '', '_master': '', '_save_summary_steps': 100, '_environment': 'local', '_session_config': None, '_keep_checkpoint_max': 40, '_keep_checkpoint_every_n_hours': 10000, '_num_ps_replicas': 0, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7efb90707e10>, '_num_worker_replicas': 0, '_model_dir': './model-k', '_task_type': None, '_tf_random_seed': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_log_step_count_steps': 100, '_is_chief': True, '_save_checkpoints_steps': None}
INFO:tensorflow:Training model for 500 steps
Instructions for updating:
Please switch to tf.train.get_global_step
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into ./model-k/model.ckpt.
INFO:tensorflow:step = 1, loss = 6.66147
INFO:tensorflow:global_step/sec: 0.750271
INFO:tensorflow:step = 101, loss = 1.13782 (133.286 sec)

INFO:tensorflow:Evaluation [161/250]
INFO:tensorflow:Evaluation [162/250]
INFO:tensorflow:Evaluation [163/250]
INFO:tensorflow:Evaluation [164/250]
INFO:tensorflow:Evaluation [165/250]
INFO:tensorflow:Evaluation [166/250]
INFO:tensorflow:Evaluation [167/250]
INFO:tensorflow:Evaluation [168/250]
INFO:tensorflow:Evaluation [169/250]
INFO:tensorflow:Evaluation [170/250]
INFO:tensorflow:Evaluation [171/250]
INFO:tensorflow:Evaluation [172/250]
INFO:tensorflow:Evaluation [173/250]
INFO:tensorflow:Evaluation [174/250]
INFO:tensorflow:Evaluation [175/250]
INFO:tensorflow:Evaluation [176/250]
INFO:tensorflow:Evaluation [177/250]
INFO:tensorflow:Evaluation [178/250]
INFO:tensorflow:Evaluation [179/250]
INFO:tensorflow:Evaluation [180/250]
INFO:tensorflow:Evaluation [181/250]
INFO:tensorflow:Evaluation [182/250]
INFO:tensorflow:Evaluation [183/250]
INFO:tensorflow:Evaluation [184/250]
INFO:tensorflow:Evaluation [185/250]
INFO:tensorflow:Evaluation [186/250]
INFO:tensorflow:Evaluation [187/250]
I

INFO:tensorflow:Evaluation [104/250]
INFO:tensorflow:Evaluation [105/250]
INFO:tensorflow:Evaluation [106/250]
INFO:tensorflow:Evaluation [107/250]
INFO:tensorflow:Evaluation [108/250]
INFO:tensorflow:Evaluation [109/250]
INFO:tensorflow:Evaluation [110/250]
INFO:tensorflow:Evaluation [111/250]
INFO:tensorflow:Evaluation [112/250]
INFO:tensorflow:Evaluation [113/250]
INFO:tensorflow:Evaluation [114/250]
INFO:tensorflow:Evaluation [115/250]
INFO:tensorflow:Evaluation [116/250]
INFO:tensorflow:Evaluation [117/250]
INFO:tensorflow:Evaluation [118/250]
INFO:tensorflow:Evaluation [119/250]
INFO:tensorflow:Evaluation [120/250]
INFO:tensorflow:Evaluation [121/250]
INFO:tensorflow:Evaluation [122/250]
INFO:tensorflow:Evaluation [123/250]
INFO:tensorflow:Evaluation [124/250]
INFO:tensorflow:Evaluation [125/250]
INFO:tensorflow:Evaluation [126/250]
INFO:tensorflow:Evaluation [127/250]
INFO:tensorflow:Evaluation [128/250]
INFO:tensorflow:Evaluation [129/250]
INFO:tensorflow:Evaluation [130/250]
I

INFO:tensorflow:Evaluation [45/250]
INFO:tensorflow:Evaluation [46/250]
INFO:tensorflow:Evaluation [47/250]
INFO:tensorflow:Evaluation [48/250]
INFO:tensorflow:Evaluation [49/250]
INFO:tensorflow:Evaluation [50/250]
INFO:tensorflow:Evaluation [51/250]
INFO:tensorflow:Evaluation [52/250]
INFO:tensorflow:Evaluation [53/250]
INFO:tensorflow:Evaluation [54/250]
INFO:tensorflow:Evaluation [55/250]
INFO:tensorflow:Evaluation [56/250]
INFO:tensorflow:Evaluation [57/250]
INFO:tensorflow:Evaluation [58/250]
INFO:tensorflow:Evaluation [59/250]
INFO:tensorflow:Evaluation [60/250]
INFO:tensorflow:Evaluation [61/250]
INFO:tensorflow:Evaluation [62/250]
INFO:tensorflow:Evaluation [63/250]
INFO:tensorflow:Evaluation [64/250]
INFO:tensorflow:Evaluation [65/250]
INFO:tensorflow:Evaluation [66/250]
INFO:tensorflow:Evaluation [67/250]
INFO:tensorflow:Evaluation [68/250]
INFO:tensorflow:Evaluation [69/250]
INFO:tensorflow:Evaluation [70/250]
INFO:tensorflow:Evaluation [71/250]
INFO:tensorflow:Evaluation [

INFO:tensorflow:global_step/sec: 0.751624
INFO:tensorflow:step = 1801, loss = 0.172619 (133.045 sec)
INFO:tensorflow:global_step/sec: 0.751569
INFO:tensorflow:step = 1901, loss = 0.0975694 (133.055 sec)
INFO:tensorflow:Saving checkpoints for 1952 into ./model-k/model.ckpt.
INFO:tensorflow:Saving checkpoints for 2000 into ./model-k/model.ckpt.
INFO:tensorflow:Loss for final step: 0.116581.
INFO:tensorflow:Evaluating model now.
INFO:tensorflow:Starting evaluation at 2018-01-13-16:48:55
INFO:tensorflow:Restoring parameters from ./model-k/model.ckpt-2000
INFO:tensorflow:Evaluation [1/250]
INFO:tensorflow:Evaluation [2/250]
INFO:tensorflow:Evaluation [3/250]
INFO:tensorflow:Evaluation [4/250]
INFO:tensorflow:Evaluation [5/250]
INFO:tensorflow:Evaluation [6/250]
INFO:tensorflow:Evaluation [7/250]
INFO:tensorflow:Evaluation [8/250]
INFO:tensorflow:Evaluation [9/250]
INFO:tensorflow:Evaluation [10/250]
INFO:tensorflow:Evaluation [11/250]
INFO:tensorflow:Evaluation [12/250]
INFO:tensorflow:Eval

...

INFO:tensorflow:Evaluation [76/250]
INFO:tensorflow:Evaluation [77/250]
INFO:tensorflow:Evaluation [78/250]
INFO:tensorflow:Evaluation [79/250]
INFO:tensorflow:Evaluation [80/250]
INFO:tensorflow:Evaluation [81/250]
INFO:tensorflow:Evaluation [82/250]
INFO:tensorflow:Evaluation [83/250]
INFO:tensorflow:Evaluation [84/250]
INFO:tensorflow:Evaluation [85/250]
INFO:tensorflow:Evaluation [86/250]
INFO:tensorflow:Evaluation [87/250]
INFO:tensorflow:Evaluation [88/250]
INFO:tensorflow:Evaluation [89/250]
INFO:tensorflow:Evaluation [90/250]
INFO:tensorflow:Evaluation [91/250]
INFO:tensorflow:Evaluation [92/250]
INFO:tensorflow:Evaluation [93/250]
INFO:tensorflow:Evaluation [94/250]
INFO:tensorflow:Evaluation [95/250]
INFO:tensorflow:Evaluation [96/250]
INFO:tensorflow:Evaluation [97/250]
INFO:tensorflow:Evaluation [98/250]
INFO:tensorflow:Evaluation [99/250]
INFO:tensorflow:Evaluation [100/250]
INFO:tensorflow:Evaluation [101/250]
INFO:tensorflow:Evaluation [102/250]
INFO:tensorflow:Evaluatio

INFO:tensorflow:Evaluation [17/250]
INFO:tensorflow:Evaluation [18/250]
INFO:tensorflow:Evaluation [19/250]
INFO:tensorflow:Evaluation [20/250]
INFO:tensorflow:Evaluation [21/250]
INFO:tensorflow:Evaluation [22/250]
INFO:tensorflow:Evaluation [23/250]
INFO:tensorflow:Evaluation [24/250]
INFO:tensorflow:Evaluation [25/250]
INFO:tensorflow:Evaluation [26/250]
INFO:tensorflow:Evaluation [27/250]
INFO:tensorflow:Evaluation [28/250]
INFO:tensorflow:Evaluation [29/250]
INFO:tensorflow:Evaluation [30/250]
INFO:tensorflow:Evaluation [31/250]
INFO:tensorflow:Evaluation [32/250]
INFO:tensorflow:Evaluation [33/250]
INFO:tensorflow:Evaluation [34/250]
INFO:tensorflow:Evaluation [35/250]
INFO:tensorflow:Evaluation [36/250]
INFO:tensorflow:Evaluation [37/250]
INFO:tensorflow:Evaluation [38/250]
INFO:tensorflow:Evaluation [39/250]
INFO:tensorflow:Evaluation [40/250]
INFO:tensorflow:Evaluation [41/250]
INFO:tensorflow:Evaluation [42/250]
INFO:tensorflow:Evaluation [43/250]
INFO:tensorflow:Evaluation [

INFO:tensorflow:Evaluation [241/250]
INFO:tensorflow:Evaluation [242/250]
INFO:tensorflow:Evaluation [243/250]
INFO:tensorflow:Evaluation [244/250]
INFO:tensorflow:Evaluation [245/250]
INFO:tensorflow:Evaluation [246/250]
INFO:tensorflow:Evaluation [247/250]
INFO:tensorflow:Evaluation [248/250]
INFO:tensorflow:Evaluation [249/250]
INFO:tensorflow:Evaluation [250/250]
INFO:tensorflow:Finished evaluation at 2018-01-13-20:09:17
INFO:tensorflow:Saving dict for global step 9500: acc = 0.746375, global_step = 9500, loss = 2.85996
INFO:tensorflow:Training model for 500 steps
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from ./model-k/model.ckpt-9500
INFO:tensorflow:Saving checkpoints for 9501 into ./model-k/model.ckpt.
INFO:tensorflow:step = 9501, loss = 0.0818028
INFO:tensorflow:global_step/sec: 0.750385
INFO:tensorflow:step = 9601, loss = 0.166556 (133.266 sec)
INFO:tensorflow:global_step/sec: 0.751551
INFO:tensorflow:step = 9701, loss = 0.136319 (133.058

INFO:tensorflow:Evaluation [184/250]
INFO:tensorflow:Evaluation [185/250]
INFO:tensorflow:Evaluation [186/250]
INFO:tensorflow:Evaluation [187/250]
INFO:tensorflow:Evaluation [188/250]
INFO:tensorflow:Evaluation [189/250]
INFO:tensorflow:Evaluation [190/250]
INFO:tensorflow:Evaluation [191/250]
INFO:tensorflow:Evaluation [192/250]
INFO:tensorflow:Evaluation [193/250]
INFO:tensorflow:Evaluation [194/250]
INFO:tensorflow:Evaluation [195/250]
INFO:tensorflow:Evaluation [196/250]
INFO:tensorflow:Evaluation [197/250]
INFO:tensorflow:Evaluation [198/250]
INFO:tensorflow:Evaluation [199/250]
INFO:tensorflow:Evaluation [200/250]
INFO:tensorflow:Evaluation [201/250]
INFO:tensorflow:Evaluation [202/250]
INFO:tensorflow:Evaluation [203/250]
INFO:tensorflow:Evaluation [204/250]
INFO:tensorflow:Evaluation [205/250]
INFO:tensorflow:Evaluation [206/250]
INFO:tensorflow:Evaluation [207/250]
INFO:tensorflow:Evaluation [208/250]
INFO:tensorflow:Evaluation [209/250]
INFO:tensorflow:Evaluation [210/250]
I

({'acc': 0.74514174, 'global_step': 10000, 'loss': 2.4922359}, [])

In [11]:
from tqdm import tqdm
# now we want to predict!
paths = glob(os.path.join(DATADIR, 'test/audio/*wav'))



def test_data_generator(data):
    def generator():
        for path in data:
            _, wav = wavfile.read(path)
            wav = wav.astype(np.float32) / np.iinfo(np.int16).max
            fname = os.path.basename(path)
            yield dict(
                sample=np.string_(fname),
                wav=wav,
            )

    return generator

test_input_fn = generator_input_fn(
    x=test_data_generator(paths),
    batch_size=hparams.batch_size, 
    shuffle=False, 
    num_epochs=1,
    queue_capacity= 10 * hparams.batch_size, 
    num_threads=1,
)

model = create_model(config=run_config, hparams=hparams)
it = model.predict(input_fn=test_input_fn)


# last batch will contain padding, so remove duplicates
submission = dict()
for t in tqdm(it):
    fname = t['sample'].decode()
    submission[fname] = t


import pandas as pd
holder = np.zeros((158538,13))
holder = pd.DataFrame(holder)
labels = ['fname','yes',
 'no',
 'up',
 'down',
 'left',
 'right',
 'on',
 'off',
 'stop',
 'go',
 'silence',
 'unknown']
holder.columns = labels
holder.fname = holder.fname.astype(str)
for n,key in enumerate(submission.keys()):
    holder.fname.values[n] = key
    for i,col in enumerate(labels[1:]):
        holder[col].values[n] = submission[key]['label'][i]
holder.index = holder.fname
holder.drop('fname',axis =1,inplace=True)
holder.to_hdf('test_fold_'+str(SELECTED_FOLD)+'_strangenet_venheads-0.75.h5', 'prob')

INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_evaluation_master': '', '_master': '', '_save_summary_steps': 100, '_environment': 'local', '_session_config': None, '_keep_checkpoint_max': 40, '_keep_checkpoint_every_n_hours': 10000, '_num_ps_replicas': 0, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7efb90707e10>, '_num_worker_replicas': 0, '_model_dir': './model-k', '_task_type': None, '_tf_random_seed': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_log_step_count_steps': 100, '_is_chief': True, '_save_checkpoints_steps': None}


0it [00:00, ?it/s]

INFO:tensorflow:Restoring parameters from ./model-k/model.ckpt-10000


158560it [20:05, 131.52it/s]


### OOF predict

In [12]:
valset_oof = []
for i in valset:
    valset_oof.append(i[2])

from tqdm import tqdm
# now we want to predict!
paths = glob(os.path.join(DATADIR, 'test/audio/*wav'))



def test_data_generator(data):
    def generator():
        for path in data:
            _, wav = wavfile.read(path)
            wav = wav.astype(np.float32) / np.iinfo(np.int16).max
            wav = tf_fix_pad_transform(wav)
            fname = path.split('/')[-2] +'/'+ path.split('/')[-1]
            yield dict(
                sample=np.string_(fname),
                wav=wav,
            )

    return generator

test_input_fn = generator_input_fn(
    x=test_data_generator(valset_oof),
    batch_size=hparams.batch_size, 
    shuffle=False, 
    num_epochs=1,
    queue_capacity= 10 * hparams.batch_size, 
    num_threads=1,
)

model = create_model(config=run_config, hparams=hparams)
it = model.predict(input_fn=test_input_fn)


# last batch will contain padding, so remove duplicates
submission = dict()
for t in tqdm(it):
    fname = t['sample'].decode()
    submission[fname] = t


import pandas as pd
holder = np.zeros((len(valset_oof),13))
holder = pd.DataFrame(holder)
labels = ['fname','yes',
 'no',
 'up',
 'down',
 'left',
 'right',
 'on',
 'off',
 'stop',
 'go',
 'silence',
 'unknown']
holder.columns = labels
holder.fname = holder.fname.astype(str)
for n,key in enumerate(submission.keys()):
    holder.fname.values[n] = key
    for i,col in enumerate(labels[1:]):
        holder[col].values[n] = submission[key]['label'][i]
holder.index = holder.fname
holder.drop('fname',axis =1,inplace=True)
holder.to_hdf('train_fold_'+str(SELECTED_FOLD)+'_strangenet_venheads-0.75.h5', 'prob')

INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_evaluation_master': '', '_master': '', '_save_summary_steps': 100, '_environment': 'local', '_session_config': None, '_keep_checkpoint_max': 40, '_keep_checkpoint_every_n_hours': 10000, '_num_ps_replicas': 0, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7efb90707e10>, '_num_worker_replicas': 0, '_model_dir': './model-k', '_task_type': None, '_tf_random_seed': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_log_step_count_steps': 100, '_is_chief': True, '_save_checkpoints_steps': None}


0it [00:00, ?it/s]

INFO:tensorflow:Restoring parameters from ./model-k/model.ckpt-10000


12832it [01:38, 130.17it/s]
